In [346]:
import numpy as np
import matplotlib.pyplot as plt

from skimage import transform as tf
from skimage import io

%matplotlib osx

# Part 1 - Warping images

## Load in and display images of a DIC map and EBSD grain boundaries

In [731]:
dicImagePath = "example_data/step5-raw.png"
ebsdImagePath = "example_data/boundaries.png"

dicImage = io.imread(dicImagePath)
ebsdImage = io.imread(ebsdImagePath)

io.imshow(dicImage)
plt.figure()
io.imshow(ebsdImage)


## Define homologous points in both maps

In [732]:
dicPoints = np.array((
    (300, 406),
    (120, 392),
    (275, 25)
))

ebsdPoints = np.array((
    (377, 494),
    (160, 474),
    (388, 65)

))

## Display maps and points to check

In [733]:
plt.figure()
plt.imshow(dicImage)
plt.scatter(x=dicPoints[:, 0], y=dicPoints[:, 1], c='y', s=60)
plt.show()

In [734]:
plt.figure()
plt.imshow(ebsdImage)
plt.scatter(x=ebsdPoints[:, 0], y=ebsdPoints[:, 1], c='y', s=60)
plt.show()

## Estimate transformation then apply to EBSD boundaries image and display

In [735]:
ebsdTransform = tf.AffineTransform()
ebsdTransform.estimate(dicPoints, ebsdPoints)

warpedEbsdImage = tf.warp(ebsdImage, ebsdTransform)

plt.figure()
plt.imshow(warpedEbsdImage)
#plt.imsave("/Users/mbcx9ma4/project/Dogbone samples/dogbone 7/Dogbone7-Map3/boundaries-warped.png", warpedEbsdImage)

In [648]:
print(ebsdTransform.params)

[[  1.21399678e+00  -1.08529972e-01   5.68641360e+01]
 [  2.36552836e-02   1.12443207e+00   3.03839953e+01]
 [  0.00000000e+00   0.00000000e+00   1.00000000e+00]]


# Part 2: Examples of using EBSD and DIC linking classes


In [1]:
#Load packages
import numpy as np
import matplotlib.pyplot as plt

from quat import Quat

import ebsd
import hrdic

#Set plot behaviour. interactive grain selection has only been tested in osx display mode
%matplotlib osx

### Load in DIC map

In [3]:
#Load in DIC map
DicFilePath = "example_data/"
DicMap = hrdic.Map(DicFilePath, "B00005.txt")

### Load in EBSD map, detect grains thena calculate grain misorientation

In [5]:
#Load in EBSD map and calculate misorientation
EbsdFilePath = "example_data/Map Data 2-DIC area"

EbsdMap = ebsd.Map(EbsdFilePath, "cubic")
EbsdMap.binData = EbsdMap.binData[::-1]  #rotate the map 180 degrees
EbsdMap.buildQuatArray()
EbsdMap.findBoundaries(boundDef = 10)
EbsdMap.findGrains(minGrainSize=10)
EbsdMap.calcGrainMisOri(calcAxis = True)    #This line takes about 20 minutes to run but is 
                                            #required to display misorientaion
#EbsdMap.calcGrainAvOris()   #If you don't run the misoriatation calculation you need to run this instead.
                            #It takes awhile aswell but not as long as the above

### Plot misorientation map

In [784]:
EbsdMap.plotMisOriMap(plotGBs=True, vMin=0, vMax=4)

### Find a grain of interest

In [6]:
EbsdMap.locateGrainID()

489


/Users/mbcx9ma4/anaconda/lib/python2.7/site-packages/matplotlib/image.py:676: UserWarning: The backend (<class 'matplotlib.backends.backend_macosx.RendererMac'>) does not support interpolation='none'. The image will be interpolated with 'nearest` mode.
  "mode." % renderer.__class__)


510
427
285
475


### Plot misorientation for isolated grain

In [778]:
EbsdMap.grainList[EbsdMap.currGrainId].plotMisOri()

In [740]:
EbsdMap.grainList[EbsdMap.currGrainId].plotMisOri(component=4)

## Link EBSD map to DIC map

### Define a crop for the DIC map

In [7]:
DicMap.setCrop(xMin=15, xMax=28, yMin=30, yMax=15)

### Define homologous points

In [8]:
#Set homologous points
#(x, y)
DicMap.homogPoints = np.array((
    (303, 408),
    (120, 392),
    (275, 25)
))

EbsdMap.homogPoints = np.array((
    (536, 776),
    (236, 790),
    (466, 160)
))

### Display map with homologous points

In [9]:
EbsdMap.plotBoundaryMap()
plt.scatter(x=EbsdMap.homogPoints[:, 0], y=EbsdMap.homogPoints[:, 1], c='y', s=60)

In [10]:
DicMap.plotMaxShear()
plt.scatter(x=DicMap.homogPoints[:, 0], y=DicMap.homogPoints[:, 1], c='y', s=60)

### Link the maps with a shift for corrections

In [11]:
DicMap.linkEbsdMap(EbsdMap)
DicMap.setEbsdShift(xShift=0, yShift=3)

### Plot max shear with boundaries

In [12]:
DicMap.plotMaxShear(plotGBs=True)

### Detect grains in the DIC map

In [13]:
DicMap.findGrains(minGrainSize=10)

### Locate a grain of interest

In [19]:
DicMap.locateGrainID(displaySelected=True)
#displaySelected will show the selected grain in a separate figure window. 
#Do not close the window and it will update as you click grains.

144
[111] inclination: 32.9
[-111] inclination: 43.9
[1-11] inclination: 73.5
[11-1] inclination: 80.5


### Plot max shear for isolated grain

In [20]:
DicMap.grainList[DicMap.currGrainId].plotMaxShear()

### Plot a histogram of it

In [750]:
plt.figure()
plt.hist(DicMap.grainList[DicMap.currGrainId].maxShearList, bins=100);

### Plot misorientation for isolated grain

In [775]:
ebsdGrainId = DicMap.ebsdGrainIds[DicMap.currGrainId]
EbsdMap.grainList[ebsdGrainId].plotMisOri()

### Plot max shear for isolated grain with slip traces

In [21]:
DicMap.grainList[DicMap.currGrainId].calcSlipTraces()
DicMap.grainList[DicMap.currGrainId].plotMaxShear(plotSlipTraces=True)

#DicMap.grainList[DicMap.currGrainId].calcSlipTraces(correctAvOri=False)
#DicMap.grainList[DicMap.currGrainId].plotMaxShear(plotSlipTraces=True)

[111] inclination: 32.9
[-111] inclination: 43.9
[1-11] inclination: 73.5
[11-1] inclination: 80.5


### Plot boundaries without crop to see rotation

In [760]:
plt.figure()

warpedBoundaries = tf.warp(-DicMap.ebsdMap.boundaries.astype(float), DicMap.ebsdTransform) > 0.1

warpedBoundaries = -warpedBoundaries.astype(int)

plt.imshow(warpedBoundaries, cmap="gray")

In [ ]:
#Error in euler angles due to rotation of frame. Could correct the orientations or move DIC to ebsd frame.
#Things to do:
#-Sort crop out
#-Negative shifts
#-Tool for setting homog points
#-Calculate slip band plane spacing using angle of planes into sample
#-Define homog points on speckle image
#-Correct euler angles after warping (done)